In [7]:
import numpy as np
from sklearn import datasets
from dbc import *

In [8]:
K = 2
L = np.ones((K,K)) - np.eye(K)
np.random.seed(10)
X_train, y_train = datasets.make_blobs(n_samples=[125 * 5, 70 * 3], n_features=2, centers=[(9.5, 10), (9.7, 9.4)],
                                       cluster_std=[[0.6, 0.6], [0.35, 0.3]], shuffle=True, random_state=None)

In [9]:
DBC_kmeans = DiscreteBayesianClassifier(discretization_method="kmeans", discretization_params={"n_clusters": 20})
DBC_kmeans.fit(X_train,y_train)
y_pred = DBC_kmeans.predict(X_train, prior=np.array([0.7,0.3]), loss_function=L)
compute_conditional_risk(y_train, y_pred, loss_function=L)

(array([0.2064    , 0.20952381]),
 array([[0.7936    , 0.2064    ],
        [0.20952381, 0.79047619]]))

In [10]:
DBC_kmeans.predict_proba(X_train, prior=np.array([0.7,0.3]), loss_function=L)

array([[0.93860511, 0.06139489],
       [0.36080998, 0.63919002],
       [1.        , 0.        ],
       ...,
       [0.27184466, 0.72815534],
       [0.36080998, 0.63919002],
       [0.85297577, 0.14702423]])

In [11]:
DBC_fcm = DiscreteBayesianClassifier(discretization_method="fcm", discretization_params={"n_clusters": 20, "fuzzifier": 1.5})
DBC_fcm.fit(X_train,y_train)
y_pred = DBC_fcm.predict(X_train, prior=np.array([0.7,0.3]), loss_function=L)
compute_conditional_risk(y_train, y_pred, loss_function=L)

(array([0.1744    , 0.29047619]),
 array([[0.8256    , 0.1744    ],
        [0.29047619, 0.70952381]]))

In [12]:
DBC_fcm.predict_proba(X_train, prior=np.array([0.7,0.3]), loss_function=L)

array([[0.92273551, 0.07726449],
       [0.39943644, 0.60056356],
       [0.99048909, 0.00951091],
       ...,
       [0.3819068 , 0.6180932 ],
       [0.39947921, 0.60052079],
       [0.89282688, 0.10717312]])